# Neurona Logística

Como se mencionó, la función de activación **perceptrón** produce una salida binaria {0,1}, basada en que si la entrada supera o no un umbral específico. Esta función escalón hace que no sea diferenciable y, ante la falta de diferenciabilidad y su naturaleza no continua hace que el perceptrón sea dificil de entrenar mediante métodos de optimización.

Una solución que habiamos dado a esta falta de diferenciabilidad es tomar la función de activación como una función lineal, talque la salida de la neurona produzca valores reales. Esto nos es muy útil para problemas de regresión, donde el valor de salida toma valores continuos; sin embargo, ¿qué sucede para problemas de clasificación, donde por su naturaleza toma valores discretos? para ello debemos tomar otra función de activación que nos permita transformar la combinación lineal de los valores de entrada y los pesos sinápticos a valores discretos.

Por ello, para esta sección se tomará en cuenta la función logística, que pertenence a una familia de funciones denominada función sigmoide. Lla función de activación logística es suave y diferenciable en todo su rango. Esta suavidad y diferenciabilidad son esenciales para el entrenamiento eficiente de redes neuronales utilizando algoritmos de optimización, además que dicha función toma valores de salida entre un rango de [0,1], lo que la hace adecuada para problemas de clasificación binaria.

Por lo tanto, podemos escribir nuestro problema de la siguiente manera:

\begin{align}
z &= w^T x + b \\
\hat{y} &= \sigma (z) = \frac{1}{1+ e^{-z}}
\end{align}

Con esto logramos cambiar la función de activación. Donde la última expresión es conocida como la función logistica, con $y^{(i)} \in [0,1]$. Para su correcto uso en problemas de clasificación debemos imponer un umbral.

# Gradiente descendente en linea

Al igual que el problema de optimización de la neurona lineal, debemos generar un error con el cual podamos optimizar el problema, para ello se continuará con el uso de la siguiente función de perdida.

$$e_1 = L(y,\hat{y}) = \frac{1}{2} (y - \hat{y})^2$$

Con esto podemos buscar el $argmin$ de la función y expresarla con sus respectivos gradientes, tanto para los pesoss sinápticos como para el sesgo:

$$∇_w L(y,\hat{y}) = - (y - \hat{y}) \left[\hat{y}(1-\hat{y})\right]x$$

$$\frac{∂ L(y,\hat{y}) }{∂ b} = - (y - \hat{y}) \left[\hat{y}(1-\hat{y})\right]$$

De este modo, podemos llegar a las siguientes ecuaciones para entrenar el algoritmo:

\begin{align}
z = w^T x + b \\
\hat{y} = \sigma (z) = \frac{1}{1+ e^{-z}} \\
w + η (y - \hat{y}) \left[\hat{y}(1-\hat{y})\right] x \rightarrow w \\
b + η (y - \hat{y}) \left[\hat{y}(1-\hat{y})\right] \rightarrow b \\
\end{align}

Con esto $\hat{y}$ nos dará probabilidades entre $[0,1]$, pero si la pasamos por un umbral nos dará un valor discreto $\left\{0,1 \right\}$, y con esto resolvemos el problema de clasificación.


# Entropia Binaria Cruzada

Otra función de costo que podemos utilzar es la siguiente, conocida como función de entropia cruzada binaria:

$$e_3 = L(y,\hat{y}) = - \left[y log(\hat{y}) + (1+y) log(1+ \hat{y})\right]$$

Si realizamos, la optimización de esta función llegaremos a una grata sorpresa, pues las decisiones de optimización seran las mismas que una función lineal:

$$∇_w L(y,\hat{y}) = - (y - \hat{y}) x$$

$$\frac{∂ L(y,\hat{y}) }{∂ b} = - (y - \hat{y}) $$

Por lo tanto, las ecuaciones de actualización quedan:

\begin{align}
z = w^T x + b \\
\hat{y} = \sigma (z) = \frac{1}{1+ e^{-z}} \\
w + η (y - \hat{y}) x \rightarrow w \\
b + η (y - \hat{y}) \rightarrow b \\
\end{align}


# Gradiente descendente por lote

Como en el problema de la neurona lineal, también podemos entrenar el modelo por lotes de información, a partir de la siguiene función de costo:

$$e_2 = \frac{1}{2 p} \sum_{i=1}^p (y^{(i)} - \hat{y}^{(i)})^2$$

Con las siguientes ecuaciones para entrenar el algoritmo:

\begin{align}
Z = w^T X + b \\
\hat{Y} = \sigma (Z) = \frac{1}{1+ e^{-Z}} \\
w + \frac{η}{p} \left[(Y - \hat{Y}) \odot \hat{Y}(1-\hat{Y})\right] X^T \rightarrow w \\
b + \frac{η}{p} \sum_{i=1}^p\left[\left(y^{(i)}-\hat{y}^{(i)}\right)\left(\hat{y}^{(i)}\left(1-\hat{y}^{(i)}\right)\right)\right] \rightarrow b \\
\end{align}

# Programando la neurona logística

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [17]:
class Neurona_logistica:
  def __init__(self, n_entrada, factor_aprendizaje=0.1):
    self.w = -1 +2 * np.random.rand(n_entrada) # se creara un vector aleatorio entre -1 y 1
    self.b = -1 +2 * np.random.rand()
    self.eta = factor_aprendizaje

  #Creando el método de prediccion, donde solo nos dara la probabilidad
  def predict_proba(self, X):
    Z = np.dot(self.w, X) + self.b
    y_est = 1 / (1 + np.exp(-Z))
    return y_est

  #Creando el umbral de 0.5
  def predict(self, X):
    Z = np.dot(self.w, X) + self.b
    y_est = 1 / (1 + np.exp(-Z))
    return 1* (y_est > 0.5)

  #Creando el metodo para entrenar por lotes (entropia cruzada)
  def fit(self, X, Y, epochs=100):
    p = X.shape[1] #sacamos el indice 1 de X, el número de patrones
    for _ in range(epochs):
      Y_est = self.predict_proba(X) #para entrenar usamos las probabilidades
      #las actualizaciones
      self.w += (self.eta/p) * np.dot ((Y-Y_est), X.T).ravel()
      self.b += (self.eta/p) *np.sum(Y-Y_est)

In [18]:
#ejemplo
X = np.array([[0,0,1,1],
              [0,1,0,1]])
Y = np.array([[0,0,1,1]])

In [19]:
#instanciamos
neurona = Neurona_logistica(2, 0.9)

In [20]:
#obtenemos sus probabilidades
neurona.predict_proba(X)

array([0.37635081, 0.54900574, 0.39420163, 0.56759266])

In [21]:
#entrenamos
neurona.fit(X,Y)

In [22]:
#observamos si convergio correctamente a sus valores reales de Y
neurona.predict_proba(X)
#donde las probabilidades estan cerca de los valores reales de Y

array([0.07670333, 0.05658144, 0.96181888, 0.94787913])